<a href="https://colab.research.google.com/github/dvschultz/ai/blob/master/NFP_Pix2PixHD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Next Frame Prediction using Pix2PixHD

By Derrick Schultz

Forked repo and tutorial based on [JC Testud’s excellent repo and Medium](https://medium.com/@jctestud/video-generation-with-pix2pix-aed5b1b69f57) article.

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

export PS1="root@5f2a453feeb5-08f0b335a0194c0894b0a1dc5a38038d: "
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
root@5f2a453feeb5:/content# root@5f2a453feeb5-08f0b335a0194c0894b0a1dc5a38038d: umount -f /content/drive || umount /content/drive; pkill -9 -x drive
umount: /content/drive: no mount point specified.
umount: /content/drive: no mount point specified.
root@5f2a453feeb5-08f0b335a0194c0894b0a1dc5a38038d: ( while `sleep 0.5`; do if [[ -d "/content/drive" && "$(ls -A /content/drive)" != "" ]]; then echo "google.colab.drive MOUNTED"; break; fi; done ) &
[1] 174
root@5f2a453feeb5-08f0b335a0194c0894b0a1dc5a38038d: cat /tmp/tmp1grpa0a8/drive.fifo | head -1 | ( /opt/google/drive/drive --features=max_parallel_push_task_instances:10,shortcut_support:true,max_operation_batch_size:15,opendir_timeout_ms:60000,enforce_single_parent:true --inet_family=IPV4_ONLY --preferences=trusted_root_certs_file_path:/opt/google/drive/roots.pe

## Install libraries and dependencies



### Run this cell if you’ve yet to install Pix2PixHD in your Google Drive

In [0]:
%cd /content/drive/My\ Drive
!mkdir nfp-colab
%cd nfp-colab
!git clone -b video https://github.com/dvschultz/pix2pixHD.git
!pip install dominate
%cd pix2pixHD/

/content/drive/My Drive


### Run this cell if you’ve already installed Pix2PixHD in Google Drive before

In [2]:
%cd /content/drive/My\ Drive/nfp-colab/pix2pixHD/
# !git pull
!pip install dominate


/content/drive/My Drive/nfp-colab/pix2pixHD


## Extract frames from video

Upload a video to either Colab or Google Drive. I’d recommend Colab for speed.

In [0]:
!python extract_frames.py -video /content/glitch-circle-white-720.mov -name glitch-circle-white_dataset -p2pdir . -width 1280 -height 736

creating the dataset structure
ffmpeg -v 16 -i /content/glitch-circle-white-720.mov -q:v 2 -vf "scale=iw*736/ih:736, crop=1280:736" /content/drive/My Drive/nfp-colab/pix2pixHD/datasets/glitch-circle-white_dataset/train_frames/frame-%06d.jpg -hide_banner
extracting the frames
[NULL @ 0x5565df82d800] Unable to find a suitable output format for '/content/drive/My'
/content/drive/My: Invalid argument


## Train your model

### Initial training

Note: if you have a large dataset, this may timeout initially (`ValueError: __len__() should return >= 0`). Give it a minute or two and run it again.

In [9]:
!python train_video.py --name twelve-white --dataroot ./datasets/twelve-white/ --save_epoch_freq 1 #--continue_train

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
dataroot: ./datasets/twelve-white/
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
fps: 24.0
gpu_ids: [0]
heat_seeking_lvl: 0
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 35
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: twelve-white
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
pstart: 1
pstop: 1
resize_or_crop: scale_width
save_epoch_freq: 

### Continue Training

In [0]:
!python train_video.py --name fuck-white --dataroot ./datasets/fuck-white/ --save_epoch_freq 1 --continue_train

#Generating Videos

In [4]:
pwd

'/content/drive/My Drive/nfp-colab/pix2pixHD'

In [0]:
!python generate_video.py --name twelve-mask --dataroot ./datasets/twelve-mask/ --fps 24 --how_many 500 --which_epoch latest

In [0]:
import os

def processFolder(folder, epoch = 10, frameCount = 240, skipCount = 1):
  files = os.listdir(folder)

  count = 0
  for f in files:
    
    if (count % skipCount == 0):
      print(f)
      # command = 'python generate_video.py --name fuck-white --dataroot ./datasets/fuck-white/ --fps 24 --how_many %i --which_epoch latest --start_from %s/%s' % ( frameCount, folder, f)
      command = 'python generate_video.py --name fuck-white --dataroot ./datasets/fuck-white/ --fps 24 --how_many %i --which_epoch %i --start_from %s/%s' % ( frameCount, epoch, folder, f)
      os.system(command)
    count += 1

processFolder('./datasets/fuck-white/train_frames',2,500,650)

frame-018504.jpg


In [0]:
!git pull

remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/dvschultz/pix2pixHD
   314f35e..68525bf  video      -> origin/video
Updating 314f35e..68525bf
Fast-forward
 generate_video.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
